## Get necessary Libraries

In [1]:
!pip install clip torch

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=0baaf48d2e43c0e98cf80dc9bbdacbf730a007429cdd184f0b0983d3abba6cf5
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [2]:
!pip install tqdm

In [21]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import drive

## Load Data

In [2]:
# Mount to drive
drive.mount("/Drive")

Mounted at /Drive


In [3]:
# Get the csv
df = pd.read_csv("/Drive/MyDrive/Detect_LLM/Copy of merged_data.csv")

In [4]:
df

,Unnamed: 0,id,product title,image_url,price,image_name
0,5,B07XSCD2R4,Maxlite 5 Softside Expandable Luggage with 4 S...,https://m.media-amazon.com/images/I/61LnBNsSBS...,144.49,B07XSCD2R4.jpg
1,6,B07MXF4G8K,"Hard Shell Carry on Luggage Airline Approved, ...",https://m.media-amazon.com/images/I/71CghLYrnA...,169.99,B07MXF4G8K.jpg
2,7,B07H515VCZ,"Maxporter II 30"" Hardside Spinner Trunk Luggag...",https://m.media-amazon.com/images/I/81f3h+YHOX...,299.99,B07H515VCZ.jpg
3,8,B08BXBCNMQ,Omni 2 Hardside Expandable Luggage with Spinne...,https://m.media-amazon.com/images/I/91eOWP4myS...,112.63,B08BXBCNMQ.jpg
4,9,B0B9K44XTS,Luggage Sets Expandable Lightweight Suitcases ...,https://m.media-amazon.com/images/I/81dsv5GrCL...,209.99,B0B9K44XTS.jpg
...,...,...,...,...,...,...
9985,9995,B0BR617B8P,Mens Athletic Workout Shorts with Compression ...,https://m.media-amazon.com/images/I/61MFjWYxa5...,34.99,B0BR617B8P.jpg
9986,9996,B07VWSP5HD,Men's Knitted Regular Fit Full Zip Cardigan Sw...,https://m.media-amazon.com/images/I/91Hm4RVlu8...,49.99,B07VWSP5HD.jpg
9987,9997,B0CD6MBV8T,Men's Jacket Windproof Qulited Bomber Jackets ...,https://m.media-amazon.com/images/I/61sVoWGbg3...,46.98,B0CD6MBV8T.jpg
9988,9998,B08XW98F22,Men's Dry Franchise Polo,https://m.media-amazon.com/images/I/51yJ8ZYPcs...,46.87,B08XW98F22.jpg


In [5]:
new_df = df.loc[df["price"] != 0.00]

In [25]:
!unzip /Drive/MyDrive/Detect_LLM/'Copy of images.zip' -d /Drive/MyDrive/Detect_LLM/

Streaming output truncated to the last 5000 lines.
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B09GCJF4M2.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B00SO5JPXQ.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B0745858SK.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B079FT6HX4.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B07FF33WXB.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B096G42H3K.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B07MCD6PPZ.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B019DLUHIQ.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B08MW56CJR.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B0018OFUO8.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B003BRGSOG.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B0BTWJ6JZJ.jpg  
  inflating: /Drive/MyDrive/Detect_LLM/content/images/B0B46TZMQX.jpg  
  inflating: /Drive/MyDriv

## Get features


In [7]:
os.environ["HUGGINGFACE_TOKEN"] = "HF TOKEN"

In [8]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
# Define transformation for images
image_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

In [10]:
# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, clip_model, clip_processor, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.clip_model = clip_model
        self.clip_processor = clip_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 5]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        description = self.data.iloc[idx, 2]
        price = self.data.iloc[idx, 4]

        # Preprocess image
        if self.transform:
            image = self.transform(image)

        # Extract image features using CLIP model
        inputs = self.clip_processor(images=image, return_tensors="pt", padding=True)
        with torch.no_grad():
            image_features = self.clip_model.get_image_features(**inputs)

        # Extract text features using CLIP model
        inputs = self.clip_processor(
            text=[description], return_tensors="pt", padding=True
        )
        with torch.no_grad():
            text_features = self.clip_model.get_text_features(**inputs)

        sample = {
            "image": image,
            "description": description,
            "price": price,
            "image_features": image_features,
            "text_features": text_features,
        }
        return sample

In [11]:
# Paths and file names
new_df.to_csv("/Drive/MyDrive/Detect_LLM/new_df.csv", index=False)
csv_file = "/Drive/MyDrive/Detect_LLM/new_df.csv"
image_folder = "/Drive/MyDrive/Detect_LLM/content/images"

# Create dataset instance
dataset = CustomDataset(
    csv_file=csv_file,
    root_dir=image_folder,
    clip_model=clip_model,
    clip_processor=clip_processor,
    transform=image_transform,
)

In [12]:
sample = dataset[0]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


In [13]:
sample

{'image': tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.9961, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.9961, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.00

In [15]:
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [16]:
# Extract features and prices
image_features = []
text_features = []
prices = []

for batch in dataloader:
    images = batch["image"]
    descriptions = batch["description"]
    batch_text_features = batch["text_features"]
    batch_image_features = batch["image_features"]
    batch_prices = batch["price"]

    # Convert text features to numpy array
    batch_text_features_np = batch_text_features.cpu().numpy()

    # Flatten image features
    batch_image_features_np = batch_image_features.cpu().numpy()

    image_features.append(batch_image_features_np)
    text_features.append(batch_text_features_np)
    prices.append(batch_prices.numpy())

image_features = np.concatenate(image_features)
text_features = np.concatenate(text_features)
prices = np.concatenate(prices)

In [17]:
# Save image features
np.save("/Drive/MyDrive/Detect_LLM/image_features.npy", image_features)

# Save text features
np.save("/Drive/MyDrive/Detect_LLM/text_features.npy", text_features)

# Save prices
np.save("/Drive/MyDrive/Detect_LLM/prices.npy", prices)

In [18]:
# Combine text features and prices
combined_features = np.concatenate((text_features, image_features), axis=1)

In [24]:
combined_features_2d = combined_features.reshape(combined_features.shape[0], -1)

In [25]:
prices

array([18.95, 18.99, 17.99, ..., 22.99, 20.93, 44.  ])

In [19]:
import xgboost as xgb

In [26]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    combined_features_2d, prices, test_size=0.2, random_state=42
)

# Initialize and train the XGBoost regressor
xgb_regressor = xgb.XGBRegressor()
xgb_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("The R2:", r2)

Mean Squared Error: 2082.661451114399
Root Mean Squared Error: 45.63618576430768
Mean Absolute Error: 20.487665991258183
The R2: 0.5092059370199682
